In [433]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [434]:
#Logistic regression
#Random forest
# KNN

#Lightgbm
# CatBoost
# Red Neuronal

In [435]:
try:
    data_contract = pd.read_csv("final_provider/contract.csv")
    data_internet = pd.read_csv("final_provider/internet.csv")
    data_personal = pd.read_csv("final_provider/personal.csv")
    data_phone    = pd.read_csv("final_provider/phone.csv")
except:
    data_contract = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com//datasets/final_provider/contract.csv")
    data_internet = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com//datasets/final_provider/internet.csv")
    data_personal = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com//datasets/final_provider/personal.csv")
    data_phone    = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com//datasets/final_provider/phone.csv")



In [436]:
data_contract.set_index('customerID', inplace=True)
data_internet.set_index('customerID', inplace=True)
data_personal.set_index('customerID', inplace=True)
data_phone.set_index('customerID', inplace=True)


In [437]:
#juntar todas las tablas
full_data = data_contract.join([data_internet, data_personal, data_phone])


In [438]:
full_data["BeginDate"] = pd.to_datetime(full_data["BeginDate"], errors='coerce')
full_data["EndDate"] = pd.to_datetime(full_data["EndDate"], errors='coerce')

full_data['BeginDate_Year'] = full_data['BeginDate'].dt.year
full_data['BeginDate_Month'] = full_data['BeginDate'].dt.month
full_data['EndDate_Year'] = full_data['EndDate'].dt.year
full_data['EndDate_Month'] = full_data['EndDate'].dt.month

full_data['BeginDate_Year'] = full_data['BeginDate_Year'].astype("str")
full_data['BeginDate_Month'] = full_data['BeginDate_Month'].astype("str")
full_data['EndDate_Year'] = full_data['EndDate_Year'].astype("str")
full_data['EndDate_Month'] = full_data['EndDate_Month'].astype("str")

# Creación de la columna objetivo (target)
full_data["isActive"] = full_data["EndDate"].apply(
    lambda x: "Active" if pd.isna(x) else "Inactive"
)

full_data = full_data.drop(columns=["BeginDate", "EndDate"], axis =1)

# Relleno de valores ausentes, al observar los datos faltantes es muy probable que todos sean por el tipo de plan contratado, no existen datos porque no han contratado un plan que pueda generar datos en esas características
full_data.fillna("No data", inplace=True)

# Reemplazamos strings vacios por ceros
full_data.replace({"MonthlyCharges": {" ": 0}, "TotalCharges": {" ": 0}}, inplace=True)






C:\Users\armod\AppData\Local\Temp\ipykernel_20368\57462041.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  full_data["EndDate"] = pd.to_datetime(full_data["EndDate"], errors='coerce')


In [439]:
full_data.sample(10)



,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,gender,SeniorCitizen,Partner,Dependents,MultipleLines,BeginDate_Year,BeginDate_Month,EndDate_Year,EndDate_Month,isActive
customerID,,,,,,,,,,,,,,,,,,,,,
5118-MUEYH,Month-to-month,Yes,Credit card (automatic),105.10,5083.55,Fiber optic,No,Yes,No,Yes,...,Female,0,Yes,No,Yes,2016,2,nan,nan,Active
9931-DCEZH,One year,Yes,Credit card (automatic),54.40,1516.6,DSL,No,Yes,No,Yes,...,Male,0,No,Yes,No,2017,10,nan,nan,Active
1576-PFZIW,Two year,No,Credit card (automatic),105.35,7511.9,Fiber optic,Yes,Yes,Yes,No,...,Male,1,Yes,No,No,2014,4,nan,nan,Active
9873-MNDKV,Month-to-month,Yes,Mailed check,44.65,472.25,DSL,No,No,No,No,...,Female,0,No,No,No,2019,3,nan,nan,Active
1384-RCUXW,Month-to-month,Yes,Mailed check,80.55,1248.9,Fiber optic,No,Yes,No,No,...,Male,0,No,No,Yes,2018,10,nan,nan,Active
6166-YIPFO,Two year,Yes,Electronic check,64.70,4746.05,DSL,Yes,Yes,Yes,Yes,...,Male,0,Yes,No,No data,2014,2,nan,nan,Active
2498-XLDZR,Two year,No,Mailed check,73.60,2316.85,DSL,Yes,Yes,Yes,Yes,...,Female,0,Yes,Yes,No,2017,6,nan,nan,Active
5647-FXOTP,Month-to-month,Yes,Electronic check,105.90,6401.25,Fiber optic,No,Yes,Yes,No,...,Female,1,Yes,No,Yes,2015,2,nan,nan,Active
6230-BSUXY,Month-to-month,Yes,Electronic check,85.00,85,Fiber optic,No,No,Yes,No,...,Female,1,No,No,No,2019,11,2019.0,12.0,Inactive


In [440]:
# for col in full_data:
#     print(full_data[col].unique())


## MODELOS BASICOS

#### LOGISTIC REGRESSION

In [441]:
cols_categ = (
    [
        "Type",
        "PaperlessBilling",
        "PaymentMethod",
        "InternetService",
        "OnlineSecurity",
        "OnlineBackup",
        "DeviceProtection",
        "TechSupport",
        "StreamingTV",
        "StreamingMovies",
        "gender",
        "SeniorCitizen",
        "Partner",
        "Dependents",
        "MultipleLines",
        "isActive",
        "BeginDate_Year",
        "BeginDate_Month",
        "EndDate_Year",
        "EndDate_Month",
    ],
)
numeric_columns = ["MonthlyCharges", "TotalCharges"]

In [442]:
def scaler_ohe(df):
    scaler = StandardScaler()
    scaler.fit(df[numeric_columns])
    data_scaled = scaler.transform(df[numeric_columns])
    df[numeric_columns]= data_scaled
    df = pd.get_dummies(df, drop_first=True)
    
    return df

In [443]:
y = full_data["isActive"]
X = full_data.drop(columns="isActive")
X = scaler_ohe(X)
X_train,X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=999)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=999)
y.value_counts()

isActive
Active      5174
Inactive    1869
Name: count, dtype: int64

In [444]:
model_logistreg = LogisticRegression()
model_logistreg.fit(X_train, y_train)
model_logistreg.predict(X_test)


array(['Active', 'Active', 'Active', ..., 'Active', 'Inactive', 'Active'],
      dtype=object)

#### RANDOM FOREST CLASSIFIER